# NLP analysis objectives
- switch to outscraper to extract full information of reviews ✓
- check if the place might have water army?
- extract the most related
- use sentiment analysis to analysis the score rather than using google score
- NLTK

# To Do:
- download a set of data with only reviews and rating or Aurther ✓
- Word2vec to show its true reviews
- sentiment analysis to measure its true value
## data preprocessing:
- stopword ✓
- lemmetize ✓
- porterstemmer (clean liked like liking into like) ✓
- embedding ✓
- train test split ✓
- word2vec after cleaning ✓
- onehotencoding y as well

# Tech issue
- maybe code a function to download necessary nlp trained model

In [1]:
import pandas as pd
from nltk.corpus import stopwords
import nltk
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.tokenize import word_tokenize
import numpy as np
import re,string
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading wordnet: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading omw-1.4: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>


False

In [2]:
df=pd.read_csv('data_references/yelp.csv')
df=df[['text','stars']]
df.head()

,text,stars
0,My wife took me here on my birthday for breakf...,5
1,I have no idea why some people give bad review...,5
2,love the gyro plate. Rice is so good and I als...,4
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5
4,General Manager Scott Petello is a good egg!!!...,5


In [3]:
def clean_data(word):
    model= stopwords.words('english')
    trim_s=WordNetLemmatizer()
    stem=PorterStemmer()
    word = ''.join([i for i in str(word) if not i.isdigit()])
    word = word_tokenize(word.lower())
    word = [i for i in set(word) if i.isalpha() and i not in string.punctuation and i not in model]
    word = [trim_s.lemmatize(i) for i in word]
    word = [stem.stem(i) for i in word]
    
    return " ".join(word)

In [4]:
df['true_review']=np.vectorize(clean_data)(df.text)

In [5]:
df.head()

,text,stars,true_review
0,My wife took me here on my birthday for breakf...,5,white fresh outsid egg best blend truffl absol...
1,I have no idea why some people give bad review...,5,decid price tri show give like calzon forev ba...
2,love the gyro plate. Rice is so good and I als...,4,good gyro select also candi plate dig rice love
3,"Rosie, Dakota, and I LOVE Chaparral Dog Park!!...",5,path surround sniff lot field let ballpark dep...
4,General Manager Scott Petello is a good egg!!!...,5,manag total thank satisfi staff like go issu c...


In [6]:
X=df['true_review']
y=df[['stars']]

In [19]:
new_y=[]
for i in df.stars:
    if i ==5:
        new_y.append([0,0,0,0,1])
    elif i ==4:
        new_y.append([0,0,0,1,0])
    elif i ==3:
        new_y.append([0,0,1,0,0])
    elif i ==2:
        new_y.append([0,1,0,0,0])
    else:
        new_y.append([1,0,0,0,0])
new_y=np.array(new_y)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, new_y, test_size=0.33, random_state=42)

In [21]:
word2vec = Word2Vec(sentences=X_train, vector_size=60, min_count=1, window=10)

In [22]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Function to convert a sentence (list of words) into a matrix representing the words in the embedding space
def embed_sentence(word2vec, sentence):
    embedded_sentence = []
    for word in sentence:
        if word in word2vec.wv:
            embedded_sentence.append(word2vec.wv[word])
        
    return np.array(embedded_sentence)

# Function that converts a list of sentences into a list of matrices
def embedding(word2vec, sentences):
    embed = []
    
    for sentence in sentences:
        embedded_sentence = embed_sentence(word2vec, sentence)
        embed.append(embedded_sentence)
        
    return embed

# Embed the training and test sentences
X_train_embed = embedding(word2vec, X_train)
X_test_embed = embedding(word2vec, X_test)

X_train_pad = pad_sequences(X_train_embed, dtype='float32', padding='post', maxlen=200)
X_test_pad = pad_sequences(X_test_embed, dtype='float32', padding='post', maxlen=200)

In [23]:
from tensorflow.keras import Sequential
from tensorflow.keras import layers

def init_model():
    model = Sequential()
    model.add(layers.Masking())
    model.add(layers.LSTM(20, activation='tanh'))
    model.add(layers.Dense(15, activation='relu'))
    model.add(layers.Dense(5, activation='softmax'))

    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])
    
    return model
model=init_model()

In [24]:
from tensorflow.keras.callbacks import EarlyStopping

es = EarlyStopping(patience=5, restore_best_weights=True)

model.fit(X_train_pad, y_train, 
          batch_size = 64,
          epochs=10,
          callbacks=[es],
          validation_data=(X_test_pad,y_test)
         )

Epoch 1/10
105/105 [==============================] - 20s 148ms/step - loss: 1.4689 - accuracy: 0.3442 - val_loss: 1.4208 - val_accuracy: 0.3561
Epoch 2/10
105/105 [==============================] - 14s 132ms/step - loss: 1.4272 - accuracy: 0.3631 - val_loss: 1.4121 - val_accuracy: 0.3606
Epoch 3/10
105/105 [==============================] - 14s 131ms/step - loss: 1.4211 - accuracy: 0.3634 - val_loss: 1.4085 - val_accuracy: 0.3858
Epoch 4/10
105/105 [==============================] - 14s 132ms/step - loss: 1.4177 - accuracy: 0.3685 - val_loss: 1.4064 - val_accuracy: 0.3800
Epoch 5/10
105/105 [==============================] - 16s 155ms/step - loss: 1.4118 - accuracy: 0.3745 - val_loss: 1.4046 - val_accuracy: 0.3882
Epoch 6/10
105/105 [==============================] - 18s 174ms/step - loss: 1.4100 - accuracy: 0.3787 - val_loss: 1.4079 - val_accuracy: 0.3779
Epoch 7/10
105/105 [==============================] - 19s 178ms/step - loss: 1.4033 - accuracy: 0.3843 - val_loss: 1.4049 - val_ac

In [ ]:

!git add NLP.ipynb
!git commit -m 'need to improve the accuracy of the model'
!git push origin master
